In [1]:
import pandas as pd
import numpy as np
import os as os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import funcionalidade as fc

In [2]:
ativos = pd.read_pickle('ativos_fatura_premiada.pkl')

In [3]:
cliques = pd.read_csv('export_retornos_infobip_fatura_turbinada.csv', usecols = ['to','send_at','opens','clicks'], parse_dates=['send_at'])

In [4]:
cliques.head()

,to,send_at,opens,clicks
0,leandrobarbosaa528@gmail.com,2023-03-23 10:45:54,NaN,NaN
1,gabriellareis.comercial@gmail.com,2023-03-23 10:45:22,NaN,NaN
2,hian.100@hotmail.com,2023-03-23 10:45:42,NaN,NaN
3,samag967@gmail.com,2023-03-17 12:38:38,NaN,NaN
4,jefersomnunes39@gmail.com,2023-03-23 10:50:40,NaN,NaN


In [5]:
len(cliques)

5391143

In [6]:
cliques['opens'].sum()

1024282.0

In [7]:
cliques['clicks'].sum()

80135.0

In [8]:
cliques['opens'] = cliques['opens'].fillna(0.0)
cliques['clicks'] = cliques['clicks'].fillna(0.0)

In [9]:
cliques = cliques.groupby('to').agg({'opens':['max'],'clicks':['max'],'send_at':['min', 'max'] })
cliques.columns = cliques.columns.droplevel(0)
cliques = cliques.reset_index()
cliques.head()

,to,max,max,min,max
0,0.9martinsmariana@gmail.com,1.0,0.0,2023-03-17 13:02:41,2023-03-30 14:24:44
1,0.9pollyane9798@gmail.com,0.0,0.0,2023-03-17 12:53:47,2023-03-30 14:21:39
2,0.notbook.0@gmail.com,1.0,0.0,2023-03-17 13:02:51,2023-03-30 14:24:48
3,00.willian@gmail.com,0.0,0.0,2023-03-17 12:53:37,2023-03-30 14:21:26
4,000.azmildo@gmail.com,0.0,0.0,2023-03-17 13:02:55,2023-03-30 14:24:40


In [10]:
cliques.columns = ['to', 'opens', 'clicks', 'send_at_min', 'send_at_max']
cliques.head()

,to,opens,clicks,send_at_min,send_at_max
0,0.9martinsmariana@gmail.com,1.0,0.0,2023-03-17 13:02:41,2023-03-30 14:24:44
1,0.9pollyane9798@gmail.com,0.0,0.0,2023-03-17 12:53:47,2023-03-30 14:21:39
2,0.notbook.0@gmail.com,1.0,0.0,2023-03-17 13:02:51,2023-03-30 14:24:48
3,00.willian@gmail.com,0.0,0.0,2023-03-17 12:53:37,2023-03-30 14:21:26
4,000.azmildo@gmail.com,0.0,0.0,2023-03-17 13:02:55,2023-03-30 14:24:40


In [11]:
#ativos.to_csv('ativos_fatura_premiada.csv')

In [12]:
ativos['grupo_comunicacao'].value_counts()

Comunicação MCC         1404826
Sem comunicação          173079
Comunicação Genérica     150575
Name: grupo_comunicacao, dtype: int64

In [46]:
ativos_v1 = ativos.query('segmento_mcc_agg != "0 - OUTROS"')


In [47]:
ativos_v1['grupo_comunicacao'].value_counts()

Comunicação MCC         772563
Sem comunicação          95183
Comunicação Genérica     85876
Name: grupo_comunicacao, dtype: int64

### Trazendo conversão

In [48]:
ativos_v1_c = fc.traz_conversao('flavia-costa', '2023-03-17', '2023-03-24', 'cpf', ativos_v1)

In [15]:
len(ativos_v1_c)

1577905

In [16]:
ativos_v1_c['converteu'].value_counts()

0.0    809177
1.0    768728
Name: converteu, dtype: int64

### Traz personas

In [17]:
import acessos as ac

In [49]:
personas = ac.df_athena_q('flavia-costa', 
                          """select 
                                ci.cpf
                                , max(email) as to
                                , max(ds_cluster_sociodemographic) as ds_cluster_sociodemographic
                                from customer_curated_zone.ca_book_cliente_cluster cl
                                left join customer_curated_zone.ca_book_cliente ci on (ci.id_customer = cl.id_customer)
                                left join growth_curated_zone.clientes gc on (gc.cpf = ci.cpf)
                                group by 1 """)

ativos_v1_p1 = ativos_v1_c.join(personas.set_index('cpf'), how = 'left', on = 'cpf')

In [50]:
ativos_v1_p = ativos_v1_p1.join(cliques.set_index('to'), how = 'left', on = 'to')

In [51]:
limite_disp = ac.df_athena_q('flavia-costa', 
                          """with base_limite_diaria as (
                                    select
                                    dt_data
                                    , cpf
                                    , limite
                                    , (limite - utilizado) as limite_disponivel
                                    , iu
                                    from credit_curated_zone.credit_iu_will
                                    where dt_data < to_date('2023-03-17','yyyy-mm-dd')
                                    union all
                                    select
                                    dt_movimentacao as dt_data 
                                    , cpf 
                                    , limite
                                    , limite_disponivel
                                    , iu
                                    from credit_curated_zone.credit_iu_pag
                                    where dt_movimentacao < to_date('2023-03-17','yyyy-mm-dd')
                                    )
                                    , base_lim_rankeada as (
                                    select *
                                    , row_number() over (partition by cpf order by cpf asc, dt_data desc, iu desc) as linha
                                    from base_limite_diaria
                                    )
                                    select 
                                    cpf
                                    , max(limite) as limite
                                    , min(limite_disponivel) as limite_disponivel
                                    , max(iu) as iu
                                    from base_lim_rankeada
                                    where linha = 1
                                    group by 1 """)

ativos_v1_p = ativos_v1_p.join(limite_disp.set_index('cpf'), how = 'left', on = 'cpf')

In [52]:
ativos_v1_p['opens'] = ativos_v1_p['opens'].fillna(0.0)
ativos_v1_p['clicks'] = ativos_v1_p['clicks'].fillna(0.0)
ativos_v1_p = ativos_v1_p.replace([np.inf, -np.inf], 0)

### Análises de Lift

In [53]:
ativos_v1_p.to_pickle('ativos_v2_conversao.pkl')

In [2]:
ativos_v1_p = pd.read_pickle('ativos_v2_conversao.pkl')

In [5]:
ativos_v1_p.pivot_table(index = 'ds_cluster_sociodemographic', columns= 'segmento_mcc_agg', values = 'converteu', aggfunc=np.mean).to_csv('personasxsegmento_mcc.csv')

In [4]:
ativos_v1_p.head()

,cpf,grupo_comunicacao,segmento_mcc_agg,nr_score_ie,yearmonth_ie,status,safra,conta_cartao,converteu,transacoes,...,transacoes_corr,to,ds_cluster_sociodemographic,opens,clicks,send_at_min,send_at_max,limite,limite_disponivel,iu
0,91908973234,Comunicação MCC,1 - SUPERMERCADO,29.713168,202302.0,Ativo,2021-09-01,NaN,0.0,NaN,...,0.0,Dannycamurca1@gmail.com,"26 - Mulher, 25 a 40 anos, Centro-Oeste e Norte",0.0,0.0,NaT,NaT,500.0,494.81,0.01
1,14258158470,Comunicação MCC,1 - SUPERMERCADO,50.899609,202302.0,Ativo,2021-10-01,NaN,0.0,NaN,...,0.0,millenyysilva22@gmail.com,"10 - Mulher, Solteira, Até 24 anos, cidade Peq...",1.0,0.0,2023-03-17 12:38:51,2023-03-30 14:26:19,200.0,105.10,0.47
2,22377212468,Comunicação MCC,1 - SUPERMERCADO,59.104737,202302.0,Ativo,2022-02-01,NaN,0.0,NaN,...,0.0,jcjccastorr8@gmail.com,"23 - Homem, 41 anos ou mais, Nordeste",2.0,0.0,2023-03-17 12:38:40,2023-03-30 14:26:20,200.0,7.50,0.96
3,16004179493,Comunicação MCC,1 - SUPERMERCADO,56.762277,202302.0,Ativo,2022-08-01,NaN,0.0,NaN,...,0.0,taynara_ramos7@hotmail.com,"10 - Mulher, Solteira, Até 24 anos, cidade Peq...",0.0,0.0,2023-03-17 12:38:41,2023-03-30 14:26:14,500.0,252.32,0.50
4,08374986514,Comunicação MCC,1 - SUPERMERCADO,68.413733,202302.0,Ativo,2022-06-01,NaN,1.0,5.0,...,5.0,renzomoises57@gmail.com,"15 - Mulher, 25 a 40 anos, Nordeste",0.0,0.0,NaT,NaT,500.0,-7.91,1.02


Análise reduzida geral

In [7]:
fc.testes_iterativos(ativos_v1_p.query('opens >= 1 | grupo == "controle"'))

UndefinedVariableError: name 'grupo' is not defined

In [57]:
ativos_v1_p.columns

Index(['cpf', 'grupo_comunicacao', 'segmento_mcc_agg', 'nr_score_ie',
       'yearmonth_ie', 'status', 'safra', 'conta_cartao', 'converteu',
       'transacoes', 'spending', 'spending_corr', 'transacoes_corr', 'to',
       'ds_cluster_sociodemographic', 'opens', 'clicks', 'send_at_min',
       'send_at_max', 'limite', 'limite_disponivel', 'iu'],
      dtype='object')

: 

In [3]:
ativos_v1_p['flag_opens'] = np.where(ativos_v1_p['opens'] >= 1, 1, 0)
ativos_v1_p['cluster_persona'] = ativos_v1_p['ds_cluster_sociodemographic'].str[0:2]

In [4]:
ativos_v1_p['grupo'] = ['controle' if x == 'Sem comunicação' else 'alvo' for x in ativos_v1_p['grupo_comunicacao']]

In [9]:
ativos_v1_p[['converteu','grupo_comunicacao', 'flag_opens']].groupby(['grupo_comunicacao', 'flag_opens']).mean()

converteu
grupo_comunicacao    flag_opens           
Comunicação Genérica 0            0.566284
                     1            0.565976
Comunicação MCC      0            0.566952
                     1            0.564630
Sem comunicação      0            0.561601
                     1            0.562264

In [8]:
ativos_v1_p[['converteu','grupo_comunicacao', 'flag_opens']].groupby(['grupo_comunicacao']).mean()

,converteu,flag_opens
grupo_comunicacao,,
Comunicação Genérica,0.566188,0.309667
Comunicação MCC,0.566288,0.286138
Sem comunicação,0.561602,0.002784


In [7]:
ativos_v1_p[['converteu','grupo_comunicacao', 'segmento_mcc_agg','flag_opens']].groupby(['segmento_mcc_agg', 'grupo_comunicacao']).mean()

converteu  flag_opens
segmento_mcc_agg            grupo_comunicacao                          
1 - SUPERMERCADO            Comunicação Genérica   0.493713    0.298136
                            Comunicação MCC        0.498256    0.287807
                            Sem comunicação        0.488766    0.002079
2 - LOJAS DE DEPARTAMENTO   Comunicação Genérica   0.504266    0.304757
                            Comunicação MCC        0.502356    0.275972
                            Sem comunicação        0.501830    0.003661
3 - SERVICOS DE ALIMENTACAO Comunicação Genérica   0.587939    0.313664
                            Comunicação MCC        0.585779    0.293628
                            Sem comunicação        0.583337    0.001880
4 - SAUDE                   Comunicação Genérica   0.619274    0.309427
                            Comunicação MCC        0.618502    0.283309
                            Sem comunicação        0.608985    0.003122
5 - LOJAS ESPECIALIZADAS    Comunicação Genérica   0.595194    0.323097
                            Comunicação MCC        0.600096    0.294905
                            Sem comunicação        0.599280    0.002881

In [11]:
ativos_v1_p[['converteu','grupo_comunicacao']].groupby(['grupo_comunicacao']).mean()

,converteu
grupo_comunicacao,
Comunicação Genérica,0.566188
Comunicação MCC,0.566288
Sem comunicação,0.561602


In [32]:
ativos_v1_p.query('opens >= 1 | grupo == "controle"')[['converteu','grupo_comunicacao']].groupby(['grupo_comunicacao']).mean()

,converteu
grupo_comunicacao,
Comunicação Genérica,0.565976
Comunicação MCC,0.564630
Sem comunicação,0.561602


In [36]:
ativos_v1_p \
.query('nr_score_ie >47.35 & nr_score_ie <= 51.52') \
[['converteu','grupo_comunicacao']].groupby(['grupo_comunicacao']).mean()

,converteu
grupo_comunicacao,
Comunicação Genérica,0.480277
Comunicação MCC,0.485039
Sem comunicação,0.491098


In [38]:
ativos_v1_p.query('nr_score_ie >47.35 & nr_score_ie <= 51.52') \
.query('opens >= 1 | grupo == "controle"') \
[['converteu','grupo_comunicacao']].groupby(['grupo_comunicacao']).mean()

,converteu
grupo_comunicacao,
Comunicação Genérica,0.461068
Comunicação MCC,0.480507
Sem comunicação,0.491098


In [39]:
ativos_v1_p[ativos_v1_p['ds_cluster_sociodemographic'].isna() == False]  \
.query('nr_score_ie > 51.52') \
.query('cluster_persona == "26" | cluster_persona == "19" | cluster_persona == "31"| cluster_persona == "12"| cluster_persona == "2" | cluster_persona == "13"') \
[['converteu','grupo_comunicacao']].groupby(['grupo_comunicacao']).mean()

,converteu
grupo_comunicacao,
Comunicação Genérica,0.590895
Comunicação MCC,0.605029
Sem comunicação,0.606164


In [40]:
ativos_v1_p[ativos_v1_p['ds_cluster_sociodemographic'].isna() == False]  \
.query('nr_score_ie > 51.52') \
.query('opens >= 1 | grupo == "controle"') \
.query('cluster_persona == "26" | cluster_persona == "19" | cluster_persona == "31"| cluster_persona == "12"| cluster_persona == "2" | cluster_persona == "13"') \
[['converteu','grupo_comunicacao']].groupby(['grupo_comunicacao']).mean()

,converteu
grupo_comunicacao,
Comunicação Genérica,0.594231
Comunicação MCC,0.602639
Sem comunicação,0.606164


In [41]:
ativos_v1_p[ativos_v1_p['ds_cluster_sociodemographic'].isna() == False] \
.query('cluster_persona.str.contains("18")') \
.query('segmento_mcc_agg.str.contains("LOJAS ESPECIALIZADAS")') \
[['converteu','grupo_comunicacao']].groupby(['grupo_comunicacao']).mean()

,converteu
grupo_comunicacao,
Comunicação Genérica,0.598784
Comunicação MCC,0.646191
Sem comunicação,0.636842


In [42]:
ativos_v1_p[ativos_v1_p['ds_cluster_sociodemographic'].isna() == False] \
.query('cluster_persona.str.contains("18")') \
.query('opens >= 1 | grupo == "controle"') \
.query('segmento_mcc_agg.str.contains("LOJAS ESPECIALIZADAS")') \
[['converteu','grupo_comunicacao']].groupby(['grupo_comunicacao']).mean()

,converteu
grupo_comunicacao,
Comunicação Genérica,0.608108
Comunicação MCC,0.618877
Sem comunicação,0.636842


In [43]:
ativos_v1_p['fx_iu'] = pd.qcut(ativos_v1_p['iu'].rank(method='first'), q=10, precision=0, labels=[1,2,3,4,5,6,7,8,9,10])
ativos_v1_p['fx_limite'] = pd.qcut(ativos_v1_p['limite'].rank(method='first'), q=10, precision=0, labels=[1,2,3,4,5,6,7,8,9,10])
ativos_v1_p['fx_limite_disponivel'] = pd.qcut(ativos_v1_p['limite_disponivel'].rank(method='first'), q=10, precision=0, labels=[1,2,3,4,5,6,7,8,9,10])

In [45]:
ativos_v1_p[['converteu','grupo_comunicacao', 'fx_limite']].groupby(['grupo_comunicacao','fx_limite']).mean()

converteu
grupo_comunicacao    fx_limite           
Comunicação Genérica 1           0.466890
                     2           0.523612
                     3           0.563897
                     4           0.554966
                     5           0.554792
                     6           0.576844
                     7           0.572731
                     8           0.593979
                     9           0.606740
                     10          0.632761
Comunicação MCC      1           0.464995
                     2           0.533190
                     3           0.562488
                     4           0.551758
                     5           0.557094
                     6           0.575417
                     7           0.582375
                     8           0.591658
                     9           0.604469
                     10          0.639630
Sem comunicação      1           0.462914
                     2           0.526691
                     3           0.552444
                     4           0.551413
                     5           0.556022
                     6           0.570548
                     7           0.578880
                     8           0.586873
                     9           0.600353
                     10          0.641277

In [46]:
ativos_v1_p[['converteu','grupo_comunicacao', 'fx_limite_disponivel']].groupby(['grupo_comunicacao','fx_limite_disponivel']).mean()

converteu
grupo_comunicacao    fx_limite_disponivel           
Comunicação Genérica 1                      0.365061
                     2                      0.442600
                     3                      0.580592
                     4                      0.610447
                     5                      0.613885
                     6                      0.611670
                     7                      0.607534
                     8                      0.615946
                     9                      0.601641
                     10                     0.608976
Comunicação MCC      1                      0.365204
                     2                      0.435587
                     3                      0.582782
                     4                      0.607027
                     5                      0.612269
                     6                      0.605958
                     7                      0.619527
                     8                      0.619579
                     9                      0.598273
                     10                     0.616090
Sem comunicação      1                      0.361571
                     2                      0.441561
                     3                      0.578305
                     4                      0.603312
                     5                      0.599937
                     6                      0.606836
                     7                      0.620660
                     8                      0.612062
                     9                      0.586725
                     10                     0.613248

In [47]:
ativos_v1_p[['converteu','grupo_comunicacao', 'fx_iu']].groupby(['grupo_comunicacao','fx_iu']).mean()

converteu
grupo_comunicacao    fx_iu           
Comunicação Genérica 1       0.489801
                     2       0.561406
                     3       0.621885
                     4       0.635041
                     5       0.638644
                     6       0.636417
                     7       0.635033
                     8       0.616766
                     9       0.446803
                     10      0.377893
Comunicação MCC      1       0.485670
                     2       0.563171
                     3       0.618644
                     4       0.641787
                     5       0.643304
                     6       0.640124
                     7       0.638715
                     8       0.615384
                     9       0.437980
                     10      0.378046
Sem comunicação      1       0.485397
                     2       0.557672
                     3       0.608210
                     4       0.632954
                     5       0.635681
                     6       0.638954
                     7       0.629128
                     8       0.607823
                     9       0.447308
                     10      0.376001

In [48]:
ativos_v1_p[['fx_iu', 'iu']].groupby('fx_iu').agg({'iu':['min', 'max']})

iu       
        min    max
fx_iu             
1     -2.67   0.11
2      0.11   0.22
3      0.22   0.32
4      0.32   0.42
5      0.42   0.54
6      0.54   0.65
7      0.65   0.79
8      0.79   0.94
9      0.94   1.00
10     1.00  10.12

In [49]:
ativos_v1_p \
.query('iu >= 0.32 & iu <= 0.79') \
[['converteu','grupo_comunicacao']].groupby(['grupo_comunicacao']).mean()

,converteu
grupo_comunicacao,
Comunicação Genérica,0.635698
Comunicação MCC,0.640395
Sem comunicação,0.633500


In [50]:
ativos_v1_p \
.query('iu >= 0.32 & iu <= 0.79') \
.query('opens >= 1 | grupo == "controle"') \
[['converteu','grupo_comunicacao']].groupby(['grupo_comunicacao']).mean()

,converteu
grupo_comunicacao,
Comunicação Genérica,0.632977
Comunicação MCC,0.640557
Sem comunicação,0.633500


In [51]:
ativos_v1_p \
.query('iu >= 0.32 & iu <= 0.79') \
[['converteu','grupo_comunicacao']].groupby(['grupo_comunicacao']).count()

,converteu
grupo_comunicacao,
Comunicação Genérica,35347
Comunicação MCC,317298
Sem comunicação,39116


In [53]:
fc.testes_iterativos(ativos_v1_p.query('nr_score_ie >47.35 & nr_score_ie <= 51.52').query('opens >= 1 | grupo == "controle"'))

          grupo converteu    spending             transacoes         
          count      mean        mean         sum       mean      sum
grupo                                                                
alvo      16393  0.478375  287.859039  2257390.58   2.799796  21956.0
controle   6740  0.491098  281.930263   933189.17   2.797885   9261.0
_____________________
Lift da conversão: -2.59%
Estatística t do teste de diferença de proporções: -1.76
P-valor do teste de diferença de proporções: 0.078463
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: 2.1%
Estatística t do teste de spending: -0.14
P-valor do teste de spending: 0.889548
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 0.07%
Estatística t do teste de frequencia: -0.99
P-valor do teste de frequencia: 0.319916
A diferença de frequência não é signifi